# Microsoft Cognitive Services
Looking at preping a CSV for transmission to Microsoft Cognitive Services.

Current limit to 1000 rows but could make use of code in the following:
https://medium.com/@jakewa/review-millions-of-records-using-python-and-azure-cognitive-services-632e86ee226b

Consider implementing http://veekaybee.github.io/2020/02/25/secrets/

## Import required packages.

In [ ]:
import pandas as pd
import json
import requests

## Read and clean csv data into pandas data frame of required format.

In [ ]:
# Read CSV data
csv_data = pd.read_csv("ExampleData.csv")
csv_data.head()

In [ ]:
df = csv_data
df["language"] = "en"
df = df.rename(columns = {"Id":"id", "ResponseText":"text"})
df = df.drop(["Question"], axis=1)
df["id"] = df.id.astype(str)

col_order = ["id", "language", "text"]
df = df[col_order]
df.head()

## Convert to JSON document for sending to Microsoft API

In [ ]:
documents = {'documents' : eval(df.to_json(orient='records', force_ascii=False)) }
display(documents)


## API Code

### User secrets

In [ ]:
# Replace the following with the base url and key from your subscription.
subscription_key = "Put Your Key Here"
base_url = "https://dobbstextanalytics.cognitiveservices.azure.com/text/analytics/v3.0/"

key_phrase_api_url = base_url + "keyPhrases"
sentiment_api_url = base_url + "sentiment"

### Key Phrases API Call

In [ ]:
# Set Headers for REST API call
headers = {"Ocp-Apim-Subscription-Key": subscription_key}

# Make request, wait for response
response = requests.post(key_phrase_api_url, headers=headers, json=documents)
key_phrases = response.json()
key_phrases_df = pd.DataFrame(key_phrases['documents'])

In [ ]:
key_phrases_df.head()

### Sentiment API Call

In [ ]:
# Make request, wait for response
response = requests.post(sentiment_api_url, headers=headers, json=documents)
sentiment = response.json()
sentiment_df = pd.DataFrame(sentiment['documents'])


In [ ]:
sentiment_df.head()

In [ ]:
#Export to csv
key_phrases_df.to_csv("KeyPhrases.csv")
sentiment_df.to_csv("Sentiment.csv")

#Power Bi works with the raw json more easily
with open('key_phrases.json', 'w') as f:
    json.dump(key_phrases,f)

with open('sentiment.json', 'w') as f:
    json.dump(sentiment,f)